In [2]:
%reload_ext autoreload
%autoreload 3
import os

import xarray as xr
from pathlib import Path
from tqdm.auto import tqdm
import torch
import torch.nn.functional as F

from sklearn.preprocessing import LabelEncoder

from torch import nn
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from dvclive.lightning import DVCLiveLogger



# force running in the project directory (e.g., via nbconvert)
if 'notebooks' in os.getcwd():
    %cd ..

In [3]:
selected_parcellations = ['dosenbach2010', 'gordon2014', 'difumo64']



In [4]:
def get_atlas_name(file_name):
    name = file_name.replace('-','_').replace('_2mm','')
    name = name.replace('difumo_','difumo')
    return name.split('_', 1)[1]

# identify subjects
subjects = xr.open_dataset('data/julia2018/timeseries_dosenbach2010.nc5', engine='h5netcdf')['subject'].values
subject_encoder = LabelEncoder().fit(subjects)
class_encoder = LabelEncoder().fit([subj[:4] for subj in subjects])


# load time-series and concatenate them along the regions dimension


ts_files = {
    get_atlas_name(f.stem): f
    for f in Path('data/julia2018/').glob('*.nc5') if get_atlas_name(f.stem) in selected_parcellations
}

X_region = []
y_subject = []
y_class = []

s = 0


for atlas_name, ts_file in tqdm(ts_files.items()):

    ds = xr.open_dataset(ts_file, engine='h5netcdf').map(lambda x: x.values)

    # time-series
    ts = torch.tensor(ds['timeseries'].values).float()  # (n_subjects, n_timepoints, n_regions)
    X_region.append(ts)

X_region = torch.cat(X_region, dim=-1)

# subjects
subjects = subject_encoder.transform(ds['subject'].values)
y_subject = torch.tensor(subjects)  # (n_subjects, n_regions)

# classes
classes = class_encoder.transform(ds['subject'].values)
y_cls = torch.tensor(classes)  # (n_subjects, n_regions)

# normalize X_region along time axis
# X_region_norm = (X_region - X_region.mean(dim=1, keepdim=True)) / X_region.std(dim=1, keepdim=True)

X_region = X_region - X_region.mean(dim=1, keepdim=True)
X_region = F.normalize(X_region, dim=1)

  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
from src.feature_extractor import FeatureExtractor
from src.seq_auto_encoder import SeqAutoEncoder

class ACNets(pl.LightningModule):
    def __init__(self, n_inputs, n_features, hidden_size, n_subjects):
        super().__init__()
        # self.example_input_array = torch.Tensor(32, 124, 32)
        self.hidden_size = hidden_size
        self.n_subjects = n_subjects

        self.feature_extractor = FeatureExtractor(n_inputs, n_features)
        self.encoder = SeqAutoEncoder(n_inputs, n_features, hidden_size)

        # classification head
        self.head_cls = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, 2),
            nn.Softmax(dim=1)
        )
        self.head_subj_cls = nn.Linear(hidden_size, n_subjects)

    def forward(self, x):

        # feature extraction
        x_features, x_recon = self.feature_extractor(x)
        x_features_recon, h = self.encoder(x_features)

        # classifications
        y_cls = self.head_cls(h)  
        y_subj = self.head_subj_cls(h)

        return x_recon, x_features, x_features_recon, y_cls, y_subj

    def training_step(self, batch, batch_idx):
        x, y_cls, y_subj  = batch
        x_recon, x_features, x_features_recon, y_cls_hat, y_subj_hat = self(x)

        loss_recon = F.mse_loss(x_recon, x)
        loss_fe_recon = F.mse_loss(x_features_recon, x_features)
        loss_cls = F.cross_entropy(y_cls_hat, y_cls)
        loss = loss_recon + .001 * loss_fe_recon + loss_cls

        accuracy = (y_cls_hat.argmax(dim=1) == y_cls).float().mean()

        self.log('train/loss_recon', loss_recon)
        self.log('train/loss_fe_recon', loss_fe_recon)
        self.log('train/loss_cls', loss_cls)
        self.log('train/loss', loss)
        self.log('train/accuracy', accuracy)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y_cls, y_subj  = batch
        x_recon, x_features, x_features_recon, y_cls_hat, y_subj_hat = self(x)

        loss_recon = F.mse_loss(x_recon, x)
        loss_fe_recon = F.mse_loss(x_features_recon, x_features)
        loss_cls = F.cross_entropy(y_cls_hat, y_cls)
        loss = loss_recon + .001 * loss_fe_recon + loss_cls

        accuracy = (y_cls_hat.argmax(dim=1) == y_cls).float().mean()

        self.log('val/loss_recon', loss_recon)
        self.log('val/loss_fe_recon', loss_fe_recon)
        self.log('val/loss_cls', loss_cls)
        self.log('val/loss', loss)
        self.log('val/accuracy', accuracy)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

class Julia2018DataModule(pl.LightningDataModule):
    def __init__(self, X, y_cls, y_subject, train_ratio=0.75, batch_size=8, shuffle=False):
        super().__init__()
        self.X = X
        self.y_cls = y_cls
        self.y_subject = y_subject
        self.train_ratio = train_ratio
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_workers = os.cpu_count()

    def prepare_data(self):
        trn_idx, val_idx = train_test_split(
            torch.arange(0, y_subject.max() + 1),
            train_size=self.train_ratio,
            stratify=self.y_cls)

        self.trn_data = torch.utils.data.TensorDataset(self.X[trn_idx], self.y_cls[trn_idx], self.y_subject[trn_idx])
        self.val_data = torch.utils.data.TensorDataset(self.X[val_idx], self.y_cls[val_idx], self.y_subject[val_idx])

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.trn_data,
                                           batch_size=self.batch_size, shuffle=self.shuffle,
                                           num_workers=self.num_workers)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_data,
                                           batch_size=self.batch_size, shuffle=self.shuffle,
                                           num_workers=self.num_workers)


datamodule = Julia2018DataModule(X_region, y_cls, y_subject, batch_size=32, shuffle=False)

n_subjects, n_timepoints, n_inputs = X_region.shape
n_features = 32
hidden_size = 32

model = ACNets(n_inputs, n_features, hidden_size, n_subjects)
# model = torch.compile(model)  # is not supported on GTX 1070

trainer = pl.Trainer(
    max_epochs=1000, accelerator='auto', log_every_n_steps=1,
    logger=[DVCLiveLogger(save_dvc_exp=True, report='md'),
            TensorBoardLogger('.')]
    )
trainer.fit(model, datamodule=datamodule)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/morteza/micromamba/envs/acnets-multihead/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /home/morteza/workspace/ACNets-MultiHead/DvcLiveLogger/dvclive_run/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type             | Params
-------------------------------------------------------
0 | feature_extractor | FeatureExtractor | 76.1 K
1 | encoder           | SeqAutoEncoder   | 18.0 K
2 | head_cls          | Sequential       | 562   
3 | head_subj_cls     | Linear           | 1.1 K 
-------------------------------------------------------
95.7 K    Trainable params
0         Non-trainable params
95.7 K    Total params
0.383     Total es

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/home/morteza/micromamba/envs/acnets-multihead/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Updating lock file 'dvc.lock'


INFO:dvc.dvcfile:Updating lock file 'dvc.lock'
INFO:dvclive:To run with DVC, add this to /home/morteza/workspace/ACNets-MultiHead/dvc.yaml:
stages:
  dvclive:
    cmd: <python my_code_file.py my_args>
    deps:
    - <my_code_file.py>

